<a href="https://colab.research.google.com/github/Hsieh121/generative_ai/blob/main/HW10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers accelerate safetensors huggingface_hub gradio --upgrade

from diffusers import StableDiffusionPipeline, UniPCMultistepScheduler
import torch
import gc
import matplotlib.pyplot as plt
import gradio as gr
import random
import os
from google.colab import userdata

In [ ]:
model_name = "stablediffusionapi/sdvn5-3dcutewave"

pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    #use_safetensors=True
).to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

In [ ]:
import requests
import torch
import random
import gc
import os

# 假設在某處獲取 API 金鑰
api_key = userdata.get('Groq')  # 或者是直接設定 '你的API金鑰'
os.environ['GROQ_API_KEY'] = api_key  # 可選，這樣設定後其他程式可以用這個環境變數

def translate_prompt_with_groq(prompt_zh, Groq):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {Groq}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "llama3-70b-8192",  # 或者其他你需要使用的模型
        "messages": [
            {
                "role": "system",
                "content": "你是一位專業可愛3D立體角色圖片生成提示詞設計師，請將使用者提供的 prompt 結合可愛3D立體的風格，並翻譯成精準且適合 Stable Diffusion 使用的英文 prompt，保留風格與細節描述。"
            },
            {
                "role": "user",
                "content": prompt_zh
            }
        ],
        "temperature": 0.3
    }

    try:
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()  # 若狀態碼不是 2xx 會拋出異常
        result = response.json()
        return result['choices'][0]['message']['content'].strip()
    except requests.exceptions.RequestException as e:
        print(f"[錯誤] 請求失敗: {e}")
        return prompt_zh  # 如果 API 請求失敗，返回原始中文 prompt

def generate_images(prompt, use_enhance, enhance_text, use_negative, negative_text,
                    use_custom_seed, custom_seed, height, width, steps, num_images):

    height = int(height)
    width = int(width)

    if height % 8 != 0 or width % 8 != 0:
        raise ValueError("高度和寬度必須是8的倍數！")

    if use_custom_seed:
        base_seed = int(custom_seed)
    else:
        base_seed = random.randint(0, 2**32 - 1)

    seeds = [base_seed + i for i in range(num_images)]

    # 翻譯正面 prompt（如果是中文而且提供了 Groq API Key）
    final_prompt = prompt
    if api_key:  # 確保提供了 API Key
        try:
            final_prompt = translate_prompt_with_groq(prompt, api_key)
        except Exception as e:
            print(f"[警告] 翻譯失敗，將使用原始 prompt: {e}")

    if use_enhance and enhance_text:
        final_prompt += ", " + enhance_text

    final_negative = negative_text if use_negative else None

    prompts = [final_prompt] * num_images
    negative_prompts = [final_negative] * num_images
    generators = [torch.Generator("cuda").manual_seed(seed) for seed in seeds]

    gc.collect()
    torch.cuda.empty_cache()

    images = []
    for i in range(num_images):
        with torch.no_grad():
            image = pipe(
                prompt=prompts[i],
                negative_prompt=negative_prompts[i] if final_negative else None,
                height=height,
                width=width,
                num_inference_steps=steps,
                guidance_scale=7.5,
                generator=generators[i]
            ).images[0]
            images.append(image)

    return images, f"使用的 random seeds: {seeds}"
print(api_key)

In [ ]:
default_enhance = "cute chibi character, 3D style, pastel colors, big sparkling eyes, soft lighting, smooth shading, toy-like textures, highly detailed, adorable proportions, cinematic render, Pixar-style, high resolution, vibrant colors"
default_negative = "ugly, deformed, low quality, blurry, grainy, out of frame, bad anatomy, extra limbs, distorted face, bad proportions, glitch, text, watermark, signature, low resolution, mutated hands, extra fingers, poorly drawn"

with gr.Blocks(css=".gradio-container {background-color: #FAFAFA; padding: 20px;} .gr-button {font-size: 18px; background: linear-gradient(to right, #667eea, #764ba2); color: white;}") as demo:
    gr.Markdown("""
    # 🎨 角色生成器
    歡迎使用！輸入提示詞、選擇設定，立即生成你的3D可愛風格的角色！
    """)

    with gr.Row():
        with gr.Column(scale=6):
            prompt = gr.Textbox(label="Prompt", placeholder="請輸入你的提示詞 (prompt)", lines=3)
            with gr.Row():
                use_enhance = gr.Checkbox(label="加強 Prompt", value=True)
                enhance_text = gr.Textbox(label="加強內容", value=default_enhance)
            with gr.Row():
                use_negative = gr.Checkbox(label="使用 Negative Prompt", value=True)
                negative_text = gr.Textbox(label="Negative Prompt 內容", value=default_negative)
            with gr.Row():
                use_custom_seed = gr.Checkbox(label="自訂 Random Seed", value=False)
                custom_seed = gr.Number(label="指定 seed (選填)", value=42)
            with gr.Row():
                height = gr.Dropdown(["512", "768", "1024"], label="高度 Height", value="512")
                width = gr.Dropdown(["512", "768", "1024"], label="寬度 Width", value="512")
            with gr.Row():
                steps = gr.Slider(10, 100, value=20, step=5, label="生成步數 (Steps)")
                num_images = gr.Slider(1, 4, step=1, value=1, label="生成張數")
            generate_btn = gr.Button("🚀 開始生成！")

        with gr.Column(scale=6):
            gallery = gr.Gallery(label="生成結果", columns=2, object_fit="contain", height="auto")
            seed_info = gr.Label(label="使用的 Random Seeds")

    generate_btn.click(
        fn=generate_images,
        inputs=[prompt, use_enhance, enhance_text, use_negative, negative_text,
                use_custom_seed, custom_seed, height, width, steps, num_images],
        outputs=[gallery, seed_info]
    )

    demo.launch(share=True, debug=True)